In [2]:
import torch
from joints2smpl.smplify import SMPLify3D
import smplx

from geometry import rot6d_to_rotmat, rotmat_to_rot6d

from tqdm import tqdm

In [3]:
smpl = smplx.SMPLLayer(model_path="models/smpl/basicmodel_neutral_lbs_10_207_0_v1.1.0.pkl")

In [8]:
joints = torch.load("results/joints.pt")
pose = torch.load("results/pose.pt")
pose_flat = pose.view(-1, 23, 3, 3)
joints_flat = joints.view(-1, 24, 3)
joints_flat.shape

torch.Size([50, 24, 3])

In [19]:
translation = torch.zeros(
    joints_flat.shape[0], 3
).requires_grad_(True)

global_orient_6d = rotmat_to_rot6d(
    torch.eye(3).unsqueeze(0).repeat(joints_flat.shape[0], 1, 1)
).requires_grad_(True)

body_pose_6d = rotmat_to_rot6d(
    pose_flat
)

In [20]:
body_pose_6d.shape

torch.Size([50, 23, 3, 2])

In [21]:
optimizer = torch.optim.Adam([translation, global_orient_6d], lr=1e-2)

In [22]:
steps = tqdm(range(1000))
for i in steps:
    optimizer.zero_grad()

    smpl_out = smpl(
        global_orient=rot6d_to_rotmat(global_orient_6d),
        body_pose=rot6d_to_rotmat(body_pose_6d),
        transl=translation,
    )

    joints_pred = smpl_out.joints[:, :24]
    loss = torch.nn.functional.mse_loss(
        joints_flat,
        joints_pred
    )

    if i % 10 == 0:
        steps.set_postfix({"loss": loss.item()})
    loss.backward()
    optimizer.step()

100%|██████████| 1000/1000 [00:24<00:00, 40.23it/s, loss=0.0014]


In [23]:
joints.shape

torch.Size([1, 1, 50, 24, 3])

In [24]:
motion = torch.cat((
    torch.ones(*joints.shape[1:-2], 1),
    translation.view(*joints.shape[1:-2], -1),
    rot6d_to_rotmat(global_orient_6d).view(*joints.shape[1:-2], -1),
    rot6d_to_rotmat(body_pose_6d).view(*joints.shape[1:-2], -1)
), dim=-1).detach().clone().cpu()

torch.save(motion, "results/motion.pt")

In [6]:
joints_flat = joints.reshape(-1, 24, 3)
print(joints_flat.shape)
smplify = SMPLify3D(smpl, batch_size=joints_flat.shape[0])

torch.Size([50, 24, 3])


In [7]:
print(joints_flat.shape)

out = smplify(
    init_pose=torch.zeros(joints_flat.shape[0], 3+23*3, 3),
    init_betas=torch.zeros(joints_flat.shape[0], 10),
    init_cam_t=torch.zeros(joints_flat.shape[0], 3),
    j3d=joints_flat
)

torch.Size([50, 24, 3])
torch.Size([50, 3, 3]) torch.Size([50, 69, 3]) torch.Size([50, 10])
torch.Size([8, 69]) torch.Size([50, 69, 3])


RuntimeError: The size of tensor a (3) must match the size of tensor b (69) at non-singleton dimension 3